In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 1 # 0.5 for Half

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/danish/anaconda3/envs/tf/lib/python3.6/site-packages

In [3]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# CNN for the IMDB problem
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


This was developed using Python 3.6 (Anaconda) and package versions:

In [4]:
tf.__version__

'1.9.0'

In [5]:
tf.keras.__version__

'2.1.6-tf'

## Load Data

We will use a data-set consisting of 50000 reviews of movies from IMDB. Keras has a built-in function for downloading a similar data-set (but apparently half the size). However, Keras' version has already converted the text in the data-set to integer-tokens, which is a crucial part of working with natural languages that will also be demonstrated in this tutorial, so we download the actual text-data.

NOTE: The data-set is 84 MB and will be downloaded automatically.

In [6]:
#import imdb
import time

tic = time.time()
relevant_path = "/data/danish/Data/IoT_Malware_Opcodes/benign_Sequences/"
print(relevant_path)

included_extentions = ['txt']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extentions)]
print(len(file_names))

X_train = []
y_train = []
wordCounter = {}
for fn in file_names:
    #file = open('/data/danish/Data/IoT_Malware_Opcodes/benign_Sequences/'+fn, 'r')
    with open('/data/danish/Data/IoT_Malware_Opcodes/benign_Sequences/'+fn,'r') as fh:
        for line in fh:
            # Replacing punctuation characters. Making the string to lower.
            # The split will spit the line into a list.
            word_list = line.replace(',','').replace('\'','').replace('.','').lower().split()
        for word in word_list:
              # Adding  the word into the wordCounter dictionary.
            if word not in wordCounter:
                wordCounter[word] = 1
            else:
                # if the word is already in the dictionary update its count.
                wordCounter[word] = wordCounter[word] + 1
    #print('{:15}{:3}'.format('Word','Count'))
    #print('-' * 25)
    
    # printing the words and its occurrence.
    #for  (word,occurance)  in wordCounter.items(): 
    #    print('{:15}{:3}'.format(word,occurance))

    data=' '.join('{}'.format(keys) for keys in wordCounter.keys())
    #data=' '.join('{}'.format(values) for values in wordCounter.values())

    X_train.append(data)
    y_train.append(0)
    
toc = time.time()
print(toc-tic)
print(len(X_train))
print(len(y_train)) 



/data/danish/Data/IoT_Malware_Opcodes/benign_Sequences/
271
4.538590431213379
271
271


In [7]:
#import imdb

tic = time.time()
relevant_path = "/data/danish/Data/IoT_Malware_Opcodes/malicious_Sequences/"
print(relevant_path)

included_extentions = ['txt']
file_names = [fn for fn in os.listdir(relevant_path)
              if any(fn.endswith(ext) for ext in included_extentions)]
print(len(file_names))


for fn in file_names:
    #file = open('/data/danish/Data/IoT_Malware_Opcodes/malicious_Sequences/'+fn, 'r')
    with open('/data/danish/Data/IoT_Malware_Opcodes/malicious_Sequences/'+fn,'r') as fh:
        for line in fh:
            # Replacing punctuation characters. Making the string to lower.
            # The split will spit the line into a list.
            word_list = line.replace(',',' ').replace('\'','').replace('.','').lower().split()
        for word in word_list:
              # Adding  the word into the wordCounter dictionary.
            if word not in wordCounter:
                wordCounter[word] = 1
            else:
                # if the word is already in the dictionary update its count.
                wordCounter[word] = wordCounter[word] + 1
    #print('{:15}{:3}'.format('Word','Count'))
    #print('-' * 25)
    
    # printing the words and its occurrence.
    #for  (word,occurance)  in wordCounter.items(): 
    #    print('{:15}{:3}'.format(word,occurance))

    data=' '.join('{}'.format(keys) for keys in wordCounter.keys())
    #data=' '.join('{}'.format(values) for values in wordCounter.values())

    X_train.append(data)
    y_train.append(1)
    
toc = time.time()

print(toc-tic)
print(len(X_train))
print(len(y_train)) 

/data/danish/Data/IoT_Malware_Opcodes/malicious_Sequences/
280
4.791821002960205
551
551


In [11]:
x_train_text, x_test_text, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [12]:
print(len(x_train_text))
print(len(y_train)) 
print(len(x_test_text))
print(len(y_test)) 

495
495
56
56


In [13]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))


Train-set size:  495
Test-set size:   56


In [14]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [15]:
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.

In [16]:
data_text = x_train_text + x_test_text

Print an example from the training-set to see that the data looks correct.

The true "class" is a sentiment of the movie-review. It is a value of 0.0 for a negative sentiment and 1.0 for a positive sentiment. In this case the review is positive.

## Tokenizer

A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.

In [17]:
num_words = None

In [18]:
tokenizer = Tokenizer(num_words=num_words)
#tokenizer_1 = Tokenizer(2000)

The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.

Note that we fit the tokenizer on the entire data-set so it gathers words from both the training- and test-data. This is OK as we are merely building a vocabulary and want it to be as complete as possible. The actual neural network will of course only be trained on the training-set.

In [19]:
#sample_text = data_text[:10]#"This movie describes the life of somebody who grew up in the worst of circumstances but unlike many people he actually grew up to be a respectable person. Whats more is that this is a true story.<br /><br />Antwone Fisher is so innocent and yet he was abused such just because he was not white. Antwone Fisher has been married to the same women for ten years and he never fooled around with women, coke, cigars, weed, alcohol, or any of those things that are very popular in the places he was growing up. <br /><br />There is not much more to say about this movie it is excellent. The only rating I can give it is a 10/10."
#tokenizer_1.fit_on_texts(sample_text)
import time
tic = time.time()
tokenizer.fit_on_texts(data_text)
toc =time.time()
print(toc-tic)

0.2697482109069824


In [20]:
len(data_text)

551

If you want to use the entire vocabulary then set `num_words=None` above, and then it will automatically be set to the vocabulary-size here. (This is because of Keras' somewhat awkward implementation.)

In [21]:
if num_words is None:
    num_words = len(tokenizer.word_index)


In [22]:
num_words

658

In [23]:
%%time
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

CPU times: user 170 ms, sys: 4.14 ms, total: 175 ms
Wall time: 173 ms


In [24]:
#x_train_tokens[1]

For example, here is a text from the training-set:

In [25]:
#x_train_text[1]

This text corresponds to the following list of tokens:

In [26]:
#x_train_tokens[1]
#np.array(x_train_tokens[1])

We also need to convert the texts in the test-set to tokens.

In [27]:
%%time
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 21.9 ms


In [28]:
#x_test_tokens[0]

## Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [29]:
%%time
import numpy as np
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

CPU times: user 430 µs, sys: 0 ns, total: 430 µs
Wall time: 442 µs


array([624, 551, 431, 456, 640, 662, 576, 603, 616, 665, 624, 327, 660,
       549, 511, 660, 616, 446, 647, 553, 504, 625, 644, 508, 458, 623,
       445, 526, 548, 441, 660, 440, 640, 508, 536, 659, 665, 369, 542,
       648, 190, 328, 660, 355, 624, 660, 491, 452, 465, 640, 549, 542,
       514, 626, 352, 626, 647, 542, 332, 622, 374, 431, 510, 396, 457,
       532, 622, 546, 647, 550, 666, 628, 662, 510, 615, 659, 620, 528,
       628, 443, 542, 576, 602, 603, 485, 627, 660, 549, 515, 625, 623,
       647, 626, 549, 550, 576, 546, 644, 352, 625, 432, 625, 624, 625,
       646, 589, 647, 649, 355, 583, 388, 646, 628, 551, 542, 616, 510,
       487, 637, 625, 514, 365, 464, 538, 514, 647, 646, 432, 646, 355,
       337, 592, 441, 624, 575, 646, 576, 644, 603, 457, 519, 647, 457,
       355, 575, 644, 647, 319, 464, 395, 628, 622, 335, 406, 613, 628,
       233, 659, 660, 440, 536, 660, 464, 624, 591, 355, 549, 616, 355,
       326, 524, 429, 464, 372, 510, 660, 465, 660, 464, 589, 53

The average number of tokens in a sequence is:

In [30]:
np.mean(num_tokens)

552.8221415607985

The maximum number of tokens in a sequence is:

In [31]:
np.max(num_tokens)

666

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [32]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

742

This covers about 95% of the data-set.

In [33]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

1.0

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [34]:
pad = 'pre'

In [35]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [36]:
%%time
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

CPU times: user 0 ns, sys: 7.57 ms, total: 7.57 ms
Wall time: 6.38 ms


We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [37]:
x_train_pad.shape

(495, 742)

The matrix for the test-set has the same shape:

In [38]:
x_test_pad.shape

(56, 742)

For example, we had the following sequence of tokens above:

In [41]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

Helper-function for converting a list of tokens back to a string of words.

In [42]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

For example, this is the original text from the data-set:

In [60]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import numpy as np

In [57]:
num_folds = 10

In [58]:
# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

In [63]:
targets

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [61]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [62]:
    
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    model = Sequential()

    embedding_size = 10

    model.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='layer_embedding_1DCNN'))

    # create the model

    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    
    history = model.fit(inputs[train],targets[train],
              validation_data=(inputs[test],targets[test]), epochs=50, batch_size=128)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

## Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set.

In [ ]:
%%time
result = model.evaluate(x_test_pad, y_test)

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))

In [ ]:
import pickle
#####################################
# write Traning History to a file
output = open('History.pkl', 'wb')
pickle.dump(history.history, output)
output.close()
#####################################

In [ ]:

# read Traning History back from the file
pkl_file = open('History.pkl', 'rb')
history = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

def plot_history(history):
    loss_list = [s for s in history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.keys() if 'acc' in s and 'val' in s]
    plt.figure(figsize = (25,7))
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history[l], 'b', label='Training loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history[l], 'g', label='Testing loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('_Graph#######LOSS.png', bbox_inches='tight')
    
    
    ## Accuracy
    plt.figure(figsize = (25,7))
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history[l], 'b', label='Training accuracy (' + str(format(history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history[l], 'g', label='Testing accuracy (' + str(format(history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('_Graph#######ACC.png', bbox_inches='tight')
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
%%time
y_pred = model.predict(x=x_test_pad)
#y_pred = y_pred.T[0]

In [ ]:
y_test = np.argmax(y_test, axis=1)
y_test

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt


y_true = y_test# ground truth labels
y_probas = y_pred# predicted probabilities generated by sklearn classifier

list_fams = ['Benign','Malware']
figsize=(8,6)
title = "ROC for CNN"
title_fontsize = 12
fig, ax = plt.subplots(1, 1, figsize=figsize)
text_fontsize = 10

skplt.metrics.plot_roc(y_true, y_probas,ax=ax, plot_macro=False, plot_micro=False )

ax.set_title(title, fontsize=title_fontsize)
#ax.get_legend().remove()
ax.legend(list_fams)
plt.ylabel('True Positive Rate (Sensitivity)')
plt.xlabel('False Positive Rate (1-Specificity)')
plt.savefig("ROC for CNN.png")
plt.show()

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
import pandas as pd
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history) 

csv_file = 'IDCNN-History.csv'
with open(csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
from pycm import ConfusionMatrix

CM_1DCNN = ConfusionMatrix(y_test,y_pred)
CM_1DCNN.save_csv("CM_1DCNN",address=True)

In [ ]:
print(CM_1DCNN)

In [ ]:
model.save("1DCNN.h5")

In [ ]:
%%time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

print("weighted")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1_score)

In [ ]:
import seaborn as sns

def plot_confusion_matrix():
    
    list_fams=['Benign','Malware']
    
    cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
    
    cm=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    
    figure = plt.gcf()
    figure.set_size_inches(20, 12)
    sns.set(font_scale=1.25)
    hm = sns.heatmap(cm, cmap= "YlGnBu" ,cbar=True, annot=True, square=True,
                 annot_kws={'size': 9}, linewidth = 0.01,
                 yticklabels=list_fams, xticklabels=list_fams)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    plt.savefig('1DCNN-Matrix.png', bbox_inches='tight')
    plt.show()
    
plot_confusion_matrix()